In [75]:
!pip install torch transformers


#Using a Machine Translation Model of googletrans to Predict and Translate The Text.

In [76]:
!pip install googletrans==4.0.0-rc1


In [77]:
from googletrans import Translator, LANGUAGES

def identify_and_translate_to_english(text):
    translator = Translator()

    # Detect the language of the input text
    detected_lang = translator.detect(text).lang

    # Translate the text to English if it's not already in English
    if detected_lang != 'en':
        translated_text = translator.translate(text, src=detected_lang, dest='en')
    else:
        # If the detected language is already English, no need to translate
        translated_text = text

    return detected_lang, translated_text.text

# Example usage:
input_text = input("Please enter a statement: ")

detected_lang, translated_text = identify_and_translate_to_english(input_text)
print(f"Original text: {input_text}")
print(f"Detected language: {LANGUAGES[detected_lang]}")
print(f"Translated text (to English): {translated_text}")



Please enter a statement: ce n'était pas bon, l'histoire était ennuyeuse
Original text: ce n'était pas bon, l'histoire était ennuyeuse
Detected language: french
Translated text (to English): It was not good, the story was boring


In [78]:
!pip install transformers


#Preprocessing the Sentence

1)**STEMMING**
2)**LEMMITIZATION**
3)**STOP WORDS REMOVAL**

In [79]:
!pip install nltk

In [80]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Initialize NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")

def preprocess_text(text, lowercasing=True,stemming=False, lemmatization=False):
    # Lowercasing
    if lowercasing:
        text = text.lower()

    # Tokenization (if not done already)
    tokens = text.split()



    # Stemming
    if stemming:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]

    # Lemmatization
    if lemmatization:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]


    processed_text = " ".join(tokens)

    return processed_text

preprocessed_text = preprocess_text(translated_text)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Using the Pretrained BERT model to classify the text and analyse the sentiment

In [81]:
from transformers import AutoModelForSequenceClassification, AutoConfig
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy
import numpy as np
from scipy.special import softmax
import tensorflow as tf

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [82]:
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import numpy as np
from scipy.special import softmax

MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def sentiment_labels(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    predicted_label = config.id2label[ranking[0]]

    # Mapping label_0 to "positive"
    if predicted_label == "label_0":
        return "positive"
    return predicted_label


sentiment_label = sentiment_labels(preprocessed_text)
print(f"Sentiment Label: {sentiment_label}")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sentiment Label: negative
